In [1]:
import vectorbt as vbt
import os
import sys
import numpy as np
import pandas as pd
from numba import njit
from plotly.subplots import make_subplots

from strategies.SellOff.SellOff import simulate_lrs, ENTRY_SIGNALS

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lib.utils import where_true_set_series, ohlcv_csv_to_df, LR, shift_np

In [2]:
file = f"/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/270000/Binance_ADAUSDT_minute.csv"
_, ohlcv = ohlcv_csv_to_df(file)
# Tomamos el close como precio.
close = ohlcv["Close"]
volume = ohlcv["Volume"]
del ohlcv

portfolio_kwargs = dict(
    direction='longonly',  # Solo long, no se shortea
    freq='m',
    size=np.inf,  # Se compra y vende toda la caja en cada operación.
    fees=0.001,
    max_logs=0,
)
thlds = np.linspace(0.5, 1.5, 3, endpoint=True)
lag = 10

lr_ind = LR.run(close)

# multi-procesamos
shifted_lr=shift_np(lr_ind.lr.to_numpy(), 1)
shifted_vol=shift_np(volume.to_numpy(), 1)

In [3]:
resps = {}
for thld in thlds:
    resps[thld] = ENTRY_SIGNALS.run(lr=lr_ind.lr, shifted_lr=shifted_lr,
                                vol=volume, shifted_vol=shifted_vol,
                                lag=lag, lr_thld=-thld, vol_thld=thld,
                                short_name="signals")

In [4]:
def plot_series_vs_scatters(series_list: list, booleans_list):
    index = None
    series = series_list.pop(0)
    fig = series.vbt.plot()
    while len(series_list):
        series = series_list.pop(0)
        if not isinstance(series, pd.Series):
            series = pd.Series(series, index=index, copy=True)
        elif index is None:
            index = series.index
        series.vbt.plot(fig=fig)
    i = 1
    for scatter in booleans_list:
        if not isinstance(scatter, pd.Series):
            scatter = pd.Series(scatter, index=index, copy=True)
        elif index is None:
            index = series.index
        scatter = where_true_set_series(series, scatter)
        scatter.name = i
        i += 1
        fig = scatter.vbt.scatterplot(fig=fig)
    return fig

In [5]:
for k, signals in resps.items():
    firsts = 5000
    plot_series_vs_scatters(
        [lr_ind.lr.head(firsts)],
        [signals.entries.head(firsts), signals.exits.head(firsts)]
    ).write_html(f"/Users/pilo/development/itba/pf/vectorbt-yaats/strategies/SellOff/plots/{k}.html")